In [31]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [32]:
from callbacks import get_callbacks

In [33]:
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.models import load_model
from tensorboard import notebook

In [34]:
from keras import models
from keras import layers
from keras import optimizers

In [35]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

In [36]:
def save_class_indices(generator, filename):
    class_indices_opp = {v:k for k, v in generator.class_indices.items()}
    
    def save_obj(obj, name ):
        with open(name + '.pkl', 'wb') as f:
            pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
    save_obj(class_indices_opp, filename)

In [37]:
def get_training_directories(base_dir):
    train_dir = os.path.join(base_dir, 'train')
    validation_dir = os.path.join(base_dir, 'val')
    test_dir = os.path.join(base_dir, 'test')
    return train_dir, validation_dir, test_dir

In [38]:
base_dir = 'data/categories_castle_cafe/categories_blurred_3_or_2_class_removed'
no_cats = 38

In [39]:
BATCH_SIZE = 2

In [40]:
def get_cnn_data_generators():
    train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    return train_datagen, test_datagen
    

In [41]:
def get_image_batches(train_datagen, test_datagen, batch_size):
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

    validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
    
    # TODO: Test generator
    return train_generator, validation_generator

In [42]:
def get_conv_base(cnn):
    if cnn == 'vgg16':
        conv_base = VGG16(weights='imagenet',
                           include_top=False,
                           input_shape=(150, 150, 3))
    elif cnn == 'inception':
        conv_base = InceptionV3(weights='imagenet',
                                 include_top=False,
                                 input_shape=(150, 150, 3))
    elif cnn == 'xception':
        conv_base = Xception(weights='imagenet',
                             include_top=False,
                             input_shape=(150, 150, 3))
    else:
        raise ValueError(f'Unknown pre-trained CNN. Got {cnn} whereas vgg16, inception or exception is expected.')
    
    return conv_base

Complete the first 3 steps:

    1) Add your custom network on top of an already trained base network.
    2) Freeze the base network.
    3) Train the part you added.

In [43]:
def get_cnn_model(pretrained_cnn, no_cats):
    conv_base = get_conv_base(pretrained_cnn)
    conv_base.trainable = False
    
    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(Dense(no_cats, activation='softmax'))

    print(model.summary())
    print(len(model.trainable_weights))
    return model

In [44]:
def build_compile_cnn(cnn, no_cats):
    model = get_cnn_model(cnn, no_cats)
    model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
                  loss='categorical_crossentropy',
                  metrics=['acc'])
    return model

In [ ]:
train_dir, validation_dir, test_dir = get_training_directories(base_dir)

train_datagen, test_datagen = get_cnn_data_generators()

train_generator, validation_generator = get_image_batches(train_datagen, test_datagen, BATCH_SIZE)

save_class_indices(validation_generator, 'class_indices_cafe')

model = build_compile_cnn('vgg16', no_cats)

callbacks = get_callbacks('vgg16_first_go', model, train_generator, validation_generator)

history = model.fit_generator(train_generator,
#                               steps_per_epoch=train_generator.samples/train_generator.batch_size,
                              steps_per_epoch=1,
                              epochs=300,
                              verbose=1,
                              validation_data=validation_generator,
                              callbacks=callbacks
                             )

Found 114 images belonging to 38 classes.
Found 38 images belonging to 38 classes.
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 38)                9766      
Total params: 16,821,862
Trainable params: 2,107,174
Non-trainable params: 14,714,688
_________________________________________________________________
None
4


/usr/local/lib/python3.6/site-packages/keras/callbacks/tensorboard_v2.py:92: UserWarning: The TensorBoard callback `batch_size` argument (for histogram computation) is deprecated with TensorFlow 2.0. It will be ignored.
  warnings.warn('The TensorBoard callback `batch_size` argument '
/usr/local/lib/python3.6/site-packages/keras/callbacks/tensorboard_v2.py:97: UserWarning: The TensorBoard callback does not support gradients display when using TensorFlow 2.0. The `write_grads` argument is ignored.
  warnings.warn('The TensorBoard callback does not support '


Epoch 1/300
1/1 [==============================] - 6s 6s/step - loss: 4.5092 - acc: 0.0000e+00 - val_loss: 3.8781 - val_acc: 0.0263
Epoch 2/300
1/1 [==============================] - 4s 4s/step - loss: 3.4585 - acc: 0.5000 - val_loss: 3.8750 - val_acc: 0.0263


Steps: 4-5

    4) Unfreeze some layers in the base network.
    5) Jointly train both these layers and the part you added.

In [ ]:
model = load_model('castle_30_vgg_features_with_data_augmentation.h5')

In [ ]:
len(model.trainable_weights)

If loaded in 

In [ ]:
# set_trainable = False
# for layer in model.get_layer("vgg16")._layers:
#     layer.trainable = False
#     if layer.name == 'block5_conv1':
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False

In [ ]:
# len(model.trainable_weights)

If using conv_base:

In [ ]:
conv_base.trainable = True

In [ ]:
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
len(model.trainable_weights)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit_generator(train_generator,
                    epochs=70,
                    validation_data=validation_generator,
#                     callbacks=[es]
                             )

In [ ]:
model.save('castle_cafe_38_fine_tuned.h5')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
def smooth_curve(points, factor=0.8):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

plt.plot(epochs, smooth_curve(acc), 'bo', label='Smoothed training acc')
plt.plot(epochs, smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, smooth_curve(loss), 'bo', label='Smoothed training loss')
plt.plot(epochs, smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Checking predictions

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
pred_probs = model.predict_generator(validation_generator)

In [ ]:
pred_probs

In [ ]:
validation_generator.classes

In [ ]:
preds = {'first': [], 'second': []}
for i in pred_probs:
    indices = i.argsort()[-3:][::-1]
    preds['first'].append(indices[0])
    preds['second'].append(indices[1])

In [ ]:
# labels = []
# for i in validation_generator.classes:
#     labels.append(np.argmax(i))

In [ ]:
labels = list(validation_generator.classes)

In [ ]:
preds

In [ ]:
labels

In [ ]:
# diffs = [a_i - b_i for a_i, b_i in zip(preds['first'], labels)]

In [ ]:
val_class_indices_opp = {v:k for k, v in validation_generator.class_indices.items()}

In [ ]:
val_class_indices = validation_generator.class_indices

In [ ]:
def get_pred_cat(cat, rank):
    category_index_model = val_class_indices[str(cat)]
#     print('category_index_model: ', category_index_model)
    index_in_labels = labels.index(category_index_model)
#     print('index_in_labels: ', index_in_labels)
#     diff_val = diffs[index_in_labels]
#     print('diff: ', diff_val)
    pred = preds[rank][index_in_labels]
    pred_category = val_class_indices_opp[pred]
    return int(pred_category)
#     print('pred_category: ', pred_category)
    # would like to know the probability here - could then calibrate

In [ ]:
def get_filenames(mypath):
    onlyfiles = [f for f in os.listdir(mypath) if not f.startswith('.')]
    return onlyfiles

In [ ]:
def get_photo(category, number, group):
    if group=='test':
        category_paths = [f'{base_dir}/test/{i}/' for i in range(1, 38) if os.path.exists(f'{base_dir}/test/{i}/')]
        try:
            test_img = os.path.join(category_paths[category-1], get_filenames(category_paths[category-1])[number-1])
            img = load_img(test_img)
        except IndexError:
            img = load_img('data/unknown_route.jpg')
    elif group=='train':
        category_paths = [f'{base_dir}/train/{i}/' for i in range(1, 38) if os.path.exists(f'{base_dir}/test/{i}/')]
        test_img = os.path.join(category_paths[category-1], get_filenames(category_paths[category-1])[number-1])
        img = load_img(test_img)        
    return img

In [ ]:
fig, ax = plt.subplots(no_cats, 3, figsize=(15,80))
for i in range(0, no_cats):
    ax[i, 0].imshow(get_photo(i+1, 1, 'train'))
    ax[i, 1].imshow(get_photo((get_pred_cat(i+1, 'first')), 1, 'test'))
    ax[i, 2].imshow(get_photo((get_pred_cat(i+1, 'second')), 1, 'test'))

Checking on test

In [ ]:
model = load_model('castle_30_vgg_fine_tuned.h5')

In [ ]:
test_dir = os.path.join(base_dir, 'test2')

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

In [ ]:
# val_class_indices

In [ ]:
pred_probs = model.predict_generator(test_generator)

In [ ]:
preds = {'first': [], 'second': []}
for i in pred_probs:
    indices = i.argsort()[-3:][::-1]
    preds['first'].append(indices[0])
    preds['second'].append(indices[1])

In [ ]:
labels = list(test_generator.classes)

In [ ]:
labels

In [ ]:
preds['first']

In [ ]:
preds['second']

In [ ]:
5/7

In [ ]:
# fig, ax = plt.subplots(no_cats, 3, figsize=(15,80))
# for i in [range(0, no_cats)]:
#     ax[i, 0].imshow(get_photo(i+1, 1, 'train'))
#     ax[i, 1].imshow(get_photo((get_pred_cat(i+1, 'first')), 1, 'test2'))
#     ax[i, 2].imshow(get_photo((get_pred_cat(i+1, 'second')), 1, 'test2'))